In [1]:
import random
import numpy as np
import copy

In [2]:
all_cpu_burst=0
def bursttime(n):
    burst=[]
    for i in range(n):
        x=random.randint(1,40)
        all_cpu_burst+=x
        burst.append(x)
    return burst    

In [3]:
def print_all_burst(all_burst):
    print("burst time - i/o time - burst time  ")
    for i in range(n):
        print(all_burst[i])

    

In [27]:

sequence=[]   
first_response=[]
all_burst=[]

pool=set()
prompt="no. of processes: "
n=int(input(prompt))
for process in range(n):
    p=bursttime(2*random.randint(1,4))   # cpu/io cycles can be adjust here
    all_burst.append(p)
    pool.add(process)

all_burst_copy= copy.deepcopy(all_burst)
print_all_burst(all_burst)  
print("processes in pool:\n",pool)  


no. of processes: 15
burst time - i/o time - burst time  
[30, 39, 18, 34]
[34, 21]
[35, 34, 37, 8, 4, 35]
[13, 9, 17, 7, 22, 33]
[38, 33]
[37, 27, 30, 23]
[3, 37, 30, 6]
[24, 3, 2, 27]
[38, 6, 31, 39]
[36, 21, 6, 3, 1, 12, 24, 21]
[15, 6, 20, 16, 22, 13, 1, 29]
[39, 5, 28, 40, 22, 3, 13, 11]
[8, 6]
[8, 38, 21, 38, 38, 24, 7, 16]
[12, 40, 34, 5, 24, 20]
processes in pool:
 {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}


In [5]:
def update_process(all_burst, process_id):
    cpu_time=0
    for i in range(len(all_burst[process_id])):
        if all_burst[process_id][i]!=0:
            cpu_time= all_burst[process_id][i]   # cpu time needed this time 
            all_burst[process_id][i]= 0
            all_burst[process_id][i+1]= 0
            break    
        
    return all_burst, cpu_time
    

In [6]:
def update_pool(pool, all_burst):
    no_of_process=len(all_burst)
    for i in range(no_of_process):
        if np.sum(np.array(all_burst[i]))==0 and i in pool:
            pool.remove(i)
        if np.sum(np.array(all_burst[i]))!=0 and i not in pool:
            pool.add(i)
            
    
    return pool

In [7]:

def multiprogramming(all_burst, pool, first_response, sequence,complition_time):
    cpu_time=0
    
    while len(pool)!=0:
        
        process_id=pool.pop()  #pops one element from the pool
        
        if process_id not in sequence:
            sequence.append(process_id)  #will give the sequence in which the process got the chance to get cpu for first time 
            first_response.append(cpu_time)  #record the first response time
        
        all_burst, c_time = update_process(all_burst, process_id)
        if all_burst[process_id][-1]==0:
            complition_time[process_id]= cpu_time + c_time
       
        
        
        cpu_time = cpu_time+ c_time
        pool= update_pool(pool, all_burst)
        
    
    
    switching_t=((all_cpu_burst/2)-1)*0.0038  # no. of switches=(len(pool)*2-1), 0.0038 ms is switching time
    efficiency= cpu_time/(cpu_time+switching_t)
    first_response=np.array(first_response)
    avg_response= np.sum(first_response)/len(all_burst)  # no. of process=len(all_burst)
    print("total cpu time to complete all processes : ",cpu_time,"ms")
    return efficiency, avg_response , first_response, sequence, complition_time
    
    

In [8]:
def waiting_time(complition_time, all_burst_copy):
    wait=[]
    i_o=[]
    
    for i in range(len(all_burst_copy)):
        s=0
        for j in range(1,len(all_burst_copy[i]),2):
            s=s+all_burst_copy[i][j]
        i_o.append(s)    
            
    for i in range(len(all_burst_copy)):
        wait.append(complition_time[i] - np.sum(np.array(all_burst_copy[i] )) + i_o[i] )
    
    return wait
    

In [28]:
complition_time=np.zeros((len(all_burst)),int)
efficiency, avg_response , first_response, sequence , complition_time= multiprogramming(all_burst, pool, first_response, sequence,complition_time)
print_all_burst(all_burst_copy)  
print("complition time of processes p0, p1, p2 ... p",n,"\n ", complition_time)
wait= waiting_time(complition_time, all_burst_copy)
print("waiting time of processes p0, p1, p2 ... p",n,"\n ", wait)
print("average waiting time:  ", np.average(np.array(wait)))
print("sequence of processes got cpu for first time\n",sequence)
print("first response at time\n ",first_response)

print("efficiency with 2ms as switching time\n ",efficiency*100,"%")
print("average response time\n ",avg_response,"ms")
print("no. of processes that can be served per second : ",(n*1000)/first_response[n-1])

total cpu time to complete all processes :  822 ms
burst time - i/o time - burst time  
[30, 39, 18, 34]
[34, 21]
[35, 34, 37, 8, 4, 35]
[13, 9, 17, 7, 22, 33]
[38, 33]
[37, 27, 30, 23]
[3, 37, 30, 6]
[24, 3, 2, 27]
[38, 6, 31, 39]
[36, 21, 6, 3, 1, 12, 24, 21]
[15, 6, 20, 16, 22, 13, 1, 29]
[39, 5, 28, 40, 22, 3, 13, 11]
[8, 6]
[8, 38, 21, 38, 38, 24, 7, 16]
[12, 40, 34, 5, 24, 20]
complition time of processes p0, p1, p2 ... p 15 
  [388  64 615 464 150 494 524 526 557 739 802 815 350 822 801]
waiting time of processes p0, p1, p2 ... p 15 
  [340, 30, 539, 412, 112, 427, 491, 500, 488, 672, 744, 713, 342, 748, 731]
average waiting time:   485.93333333333334
sequence of processes got cpu for first time
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
first response at time
  [  0  30  64  99 112 150 187 190 214 252 288 303 342 350 358]
efficiency with 2ms as switching time
  99.9865954710208 %
average response time
  195.93333333333334 ms
no. of processes that can be served per seco

In [29]:

sequence=[]   
all_burst= copy.deepcopy(all_burst_copy)


pool=set()

prompt="no. of processes: "
n=int(input(prompt))

prompt="enter the time slice in  milli sec: "
time_slice=int(input(prompt))


for process in range(n):
    #p=bursttime(3)
    #all_burst.append(p)
    pool.add(process)

  
print("burst time - i/o time - burst time  ")
for i in range(n):
    print(all_burst[i])
print("processes in pool:\n",pool)    

no. of processes: 15
enter the time slice in  milli sec: 10
burst time - i/o time - burst time  
[30, 39, 18, 34]
[34, 21]
[35, 34, 37, 8, 4, 35]
[13, 9, 17, 7, 22, 33]
[38, 33]
[37, 27, 30, 23]
[3, 37, 30, 6]
[24, 3, 2, 27]
[38, 6, 31, 39]
[36, 21, 6, 3, 1, 12, 24, 21]
[15, 6, 20, 16, 22, 13, 1, 29]
[39, 5, 28, 40, 22, 3, 13, 11]
[8, 6]
[8, 38, 21, 38, 38, 24, 7, 16]
[12, 40, 34, 5, 24, 20]
processes in pool:
 {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}


In [18]:
def update_burst_time(all_burst, process_id, time_slice):
    cpu_idle=0
    for i in range(len(all_burst[process_id])):
        if all_burst[process_id][i]>0:
            if all_burst[process_id][i]>time_slice:
                all_burst[process_id][i]= all_burst[process_id][i] - time_slice
                break
            else:
                cpu_idle= time_slice - all_burst[process_id][i]
                all_burst[process_id][i]= 0
                all_burst[process_id][i+1]= 0
                break
    
    return all_burst, cpu_idle
    
    

In [19]:
def timesharing(all_burst, pool, first_response, sequence,complition_time,time_slice):
    cpu_time=0
    total_cpu_idle=0
    
    while len(pool)!=0:
        for process_id in range(len(all_burst)):
            if process_id in pool:
                sequence.append(process_id)  #will give the sequence in which the process got the chance to run                
                all_burst, cpu_idle= update_burst_time(all_burst, process_id, time_slice)
                total_cpu_idle= total_cpu_idle + cpu_idle
                if all_burst[process_id][-1]==0:
                    complition_time[process_id]= cpu_time + time_slice
                cpu_time= cpu_time + time_slice
                pool= update_pool(pool, all_burst)
                
    switching_t=(len(sequence)-1)*0.0038  # 0.0038 ms switching time            
    efficiency=cpu_time/(cpu_time+switching_t+total_cpu_idle)
    for process in range(len(all_burst)-1):
        first_response[process+1]= first_response[process]+time_slice
    avg_response= np.sum(first_response)/len(all_burst)  # no. of process=len(all_burst)
    print("total cpu time: ",cpu_time,"ms")
    print("total cpu idle time: ",total_cpu_idle,"ms")
    return efficiency, avg_response , first_response, sequence, complition_time
    

In [30]:
first_response= np.zeros(len(all_burst),int)
complition_time=np.zeros((len(all_burst)),int)

efficiency, avg_response , first_response, sequence , complition_time= timesharing(all_burst, pool, first_response, sequence,complition_time,time_slice)
print_all_burst(all_burst_copy) 
print("complition time of processes p0, p1, p2 ... p",n,"\n ", complition_time)
wait= waiting_time(complition_time, all_burst_copy)
print("waiting time of processes p0, p1, p2 ... p",n,"\n ", wait)
print("average waiting time:  ", np.average(np.array(wait)))
print("sequence of processes got cpu for first time\n",sequence)
print("first response at time\n ",first_response)

print("efficiency with 3.8 micro sec as switching time\n ",efficiency*100,"%")
print("average response time\n ",avg_response,"ms")
print("no. of processes that can be served per second : ",(n*1000)/first_response[n-1])

total cpu time:  1000 ms
total cpu idle time:  178 ms
burst time - i/o time - burst time  
[30, 39, 18, 34]
[34, 21]
[35, 34, 37, 8, 4, 35]
[13, 9, 17, 7, 22, 33]
[38, 33]
[37, 27, 30, 23]
[3, 37, 30, 6]
[24, 3, 2, 27]
[38, 6, 31, 39]
[36, 21, 6, 3, 1, 12, 24, 21]
[15, 6, 20, 16, 22, 13, 1, 29]
[39, 5, 28, 40, 22, 3, 13, 11]
[8, 6]
[8, 38, 21, 38, 38, 24, 7, 16]
[12, 40, 34, 5, 24, 20]
complition time of processes p0, p1, p2 ... p 15 
  [ 580  450  930  780  480  790  500  510  870  940  890 1000  130  960
  970]
waiting time of processes p0, p1, p2 ... p 15 
  [532, 416, 854, 728, 442, 723, 467, 484, 801, 873, 832, 898, 122, 886, 900]
average waiting time:   663.8666666666667
sequence of processes got cpu for first time
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 0, 2, 3, 5, 8, 9, 10, 11, 13, 14, 2, 3, 5, 8, 9, 10, 11, 13, 14, 2, 3, 5, 8, 9,

In [60]:
all_burst_copy

[[5, 2, 11], [3, 19, 7], [15, 19, 8], [11, 11, 11]]

In [65]:
len(all_burst_copy[i])

3

In [24]:
from platform import python_version

print(python_version())

3.8.5


In [52]:
l= [[0, 11, 0], [0, 9, 0]]
import copy
l1=copy.copy(l)
l2= l.copy()
l3= list(l)
l4= l[:]
l5= copy.deepcopy(l)

In [54]:
l[0][0]=900
l[1][0]=111
print(l1)
print(l2)
print(l3)
print(l4)
print(l5)

[[900, 11, 0], [111, 9, 0]]
[[900, 11, 0], [111, 9, 0]]
[[900, 11, 0], [111, 9, 0]]
[[900, 11, 0], [111, 9, 0]]
[[0, 11, 0], [0, 9, 0]]


In [95]:
a=[[2,3,3,4,5,4],[0,33,0,4,0,3]]
for j in range(1,6,2):
    print(a[1][j])

33
4
3
